# Import Libraries

In [ ]:
# Need to install java first to use "read_pdf"
from tabula.io import read_pdf
import pandas as pd
from datetime import datetime
import PyPDF2
from bank_utils import *

import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
CITI_PATH = 'FOLDER/TO/SAVE/CITI/STATEMENTS/'
DBS_PATH = 'FOLDER/TO/SAVE/DBS/BANK/STATEMENT/'
DBS_CREDIT_PATH = 'FOLDER/TO/SAVE/DBS/CREDIT/STATEMENT/'
OCBC_PATH = 'FOLDER/TO/SAVE/OCBC/BANK/STATEMENT/'
FINAL_PATH = 'FOLDER/TO/SAVE/PROCESSED/TABLES/'

## CITI Credit

In [ ]:
credit_files = [i for i in os.listdir(CITI_PATH) if i[:19]=='Citibank_CreditCard']

In [ ]:
%%time

# Save all processed tables into a dictionary, of which key is the table name, value is the table
tbls = dict()
tbl_names = []
for file in credit_files:
    print(file)
    
    credit_file = CITI_PATH + file
    try:
        tbl = convert_df(credit_file, page_range=[2,3], bank='Citi')
    except: # if only one page contains table
        tbl = convert_df(credit_file, page_range=[2], bank='Citi')
        
    tbl = post_process(tbl, bank='Citi', file=file)
    tbl['Date'] = pd.to_datetime(tbl['Date'], yearfirst=True)
    if file[-24:-22] == '01': #correct the date for January report: its Dec refers to previous year
        tbl.loc[tbl['Date'].dt.month==12, 'Date'] = tbl.loc[
            tbl['Date'].dt.month==12, 'Date'] - pd.DateOffset(years=1)
    
    tbl_names.append(file[:-18])
    tbls[file[:-18]] = tbl

### Validate

In [ ]:
len(tbl_names)

In [ ]:
i = 0
print(tbl_names[i])
tbls[tbl_names[i]]

### Combine

In [ ]:
# Combine all processed tables into a single table
citi_credit_comb = pd.DataFrame()
for key in tbls.keys():
    citi_credit_comb = pd.concat([citi_credit_comb, tbls[key]], axis=0)
    
citi_credit_comb = citi_credit_comb.reset_index(drop=True)
citi_credit_comb.sort_values(by='Date', inplace=True)

In [ ]:
citi_credit_comb.info()

In [ ]:
citi_credit_comb['Date'].min(), citi_credit_comb['Date'].max()

In [ ]:
citi_credit_comb.head()

### Save

In [ ]:
citi_credit_comb.to_excel(FINAL_PATH+'Citi_Credit.xlsx')

## DBS

In [ ]:
dbs_files = [i for i in os.listdir(DBS_PATH)]
dbs_files.sort()

In [ ]:
%%time

# Save all processed tables into a dictionary, of which key is the table name, value is the table
tbls = dict()
tbl_names = []
for file in dbs_files:
    print(file)
    
    dbs_file = DBS_PATH + file
    
    # Define page_range
    tmp = open(dbs_file, 'rb')
    pdfReader = PyPDF2.PdfFileReader(tmp)
    maxpage = pdfReader.numPages
    
    tbl = convert_df(dbs_file, page_range=[i for i in range(2, maxpage)], bank='DBS')
        
    tbl = post_process(tbl, bank='DBS')
    tbl['Date'] = pd.to_datetime(tbl['Date'], yearfirst=True)
    
    tbl_names.append(file[:-4])
    tbls[file[:-4]] = tbl

### Validate

In [ ]:
len(tbls)

In [ ]:
i = 0
print(tbl_names[i])
tbls[tbl_names[i]]

### Combine

In [ ]:
# Combine all processed tables into a single table
dbs_comb = pd.DataFrame()
for key in tbls.keys():
    dbs_comb = pd.concat([dbs_comb, tbls[key]], axis=0)
    
dbs_comb = dbs_comb.reset_index(drop=True)
dbs_comb.sort_values(by='Date', inplace=True)

In [ ]:
dbs_comb.info()

In [ ]:
dbs_comb['Date'].min(), dbs_comb['Date'].max()

In [ ]:
dbs_comb.head()

### Save

In [ ]:
dbs_comb.to_excel(FINAL_PATH+'DBS.xlsx')

## DBS Credit

In [ ]:
dbs_cred_files = [i for i in os.listdir(DBS_CREDIT_PATH)]
dbs_cred_files.sort()

In [ ]:
%%time

# Save all processed tables into a dictionary, of which key is the table name, value is the table
tbls = dict()
tbl_names = []
for file in dbs_cred_files:
    print(file)
    
    dbs_file = DBS_CREDIT_PATH + file
    
    # Define page_range
    tmp = open(dbs_file, 'rb')
    pdfReader = PyPDF2.PdfFileReader(tmp)
    maxpage = pdfReader.numPages
    
    tbl = convert_df(dbs_file, page_range=[i for i in range(1, maxpage)], bank='DBS Credit')
        
    tbl = post_process(tbl, bank='DBS Credit')
    
    tbl_names.append(file[:-4])
    tbls[file[:-4]] = tbl

### Validate

In [ ]:
len(tbls)

In [ ]:
i = 0
print(tbl_names[i])
tbls[tbl_names[i]]

### Combine

In [ ]:
#Combine all processed tables into a single table
dbs_comb = pd.DataFrame()
for key in tbls.keys():
    dbs_comb = pd.concat([dbs_comb, tbls[key]], axis=0)
    
dbs_comb = dbs_comb.reset_index(drop=True)
dbs_comb.sort_values(by='Date', inplace=True)

In [ ]:
dbs_comb.info()

In [ ]:
dbs_comb['Date'].min(), dbs_comb['Date'].max()

In [ ]:
dbs_comb.head()

### Save

In [ ]:
dbs_comb.to_excel(FINAL_PATH+'DBS_Credit.xlsx')

## OCBC

In [ ]:
ocbc_files = [i for i in os.listdir(OCBC_PATH) if i[:12] == 'Consolidated']
ocbc_files.sort()

In [ ]:
%%time

# Save all processed tables into a dictionary, of which key is the table name, value is the table
tbls = dict()
tbl_names = []
error_list =  []
for file in ocbc_files:
    print(file)
    
    ocbc_file = OCBC_PATH + file
    
    # Define page_range
    try:
        tbl = convert_df(ocbc_file, page_range=[3,4], box=[3, 1, 27, 18], bank='OCBC')
        tbl = post_process(tbl, bank='OCBC', file=file)
        tbl['Date'] = pd.to_datetime(tbl['Date'], yearfirst=True)
        tbl_names.append(file[-10:-4])
        tbls[file[-10:-4]] = tbl
    except:
        print('Error in processing!!!')
        error_list.append(ocbc_file)

### Modification 
Due to Formatting Issues of Statements, the numbers of elements in date_col, desc_col, withdraw_col, depo_col are not matched for some statements 

In [ ]:
error_list

In [ ]:
filepath = error_list[0]
page = [3]
date_col, desc_col, withdraw_col, depo_col, df = mod_ocbc_df(filepath, page, box=[3, 1, 27, 30])

# Modification: Compare with original statements to correct the values
date_col[7] = '27 AUG'
date_col.remove('Co. Re')

tbl = pd.DataFrame()
tbl['Date'] = date_col
tbl['Description'] = desc_col
tbl['Withdraw'] = [float(str(i).replace(',','')) for i in withdraw_col]
tbl['Deposite'] = [float(str(i).replace(',','')) for i in depo_col]
tbl.fillna(0, inplace=True)

tbl = post_process(tbl, bank='OCBC', file=file)
tbl['Date'] = pd.to_datetime(tbl['Date'], yearfirst=True)
        
tbl_names.append(filepath.split('/')[-1][-10:-4])
tbls[filepath.split('/')[-1][-10:-4]] = tbl

### Validate

In [ ]:
len(tbls)

In [ ]:
i = 0
print(tbl_names[i])
tbls[tbl_names[i]]

### Combine

In [ ]:
# Combine all processed tables into a single table
ocbc_comb = pd.DataFrame()
for key in tbls.keys():
    ocbc_comb = pd.concat([ocbc_comb, tbls[key]], axis=0)
    
ocbc_comb = ocbc_comb.reset_index(drop=True)
ocbc_comb.sort_values(by='Date', inplace=True)

In [ ]:
ocbc_comb.info()

In [ ]:
ocbc_comb['Date'].min(), ocbc_comb['Date'].max()

In [ ]:
ocbc_comb.head()

### Save

In [ ]:
ocbc_comb.to_excel(FINAL_PATH+'OCBC.xlsx')